## Activity inside ROI

Calculates the ROI for an embryo, using the structural channel to calculate the mask.

This code relies on the naming convention in this codebase to work properly: each embryo should have a file ending in `ch1.tif` (active channel) and another one ending in `ch2.tif` (structural channel).

To speed up the analysis, the ROI can be cached with the function `roi.cache_rois`, and passed to `activity.plot_activity` through the argument `mask_path`.

In [ ]:
%load_ext autoreload
%autoreload 2
import os
from pathlib import Path

from tifffile import imread
import matplotlib.pyplot as plt

from pasnascope import roi, activity

In [13]:
project_dir = Path(os.getcwd()).parent
img_dir = os.path.join(project_dir, 'data', 'embryos')

# All structural channel movies end with the suffix ch2
active = [f for f in sorted(os.listdir(img_dir)) if f.endswith('ch1.tif')]
struct = [f for f in sorted(os.listdir(img_dir)) if f.endswith('ch2.tif')]

### Percentual error caused by downsampling

Percentual error caused by downsampling. The error is calculated by comparing a given measure with the activity calculated with a ROI that changes for every slice of the image.

In [ ]:
i = 2
print(active[i])
active_img = imread(os.path.join(img_dir,active[i]))
struct_img = imread(os.path.join(img_dir,struct[i]))

mask = roi.get_roi(struct_img)
activity_window_10 = activity.get_activity(active_img, mask)

mask = roi.get_roi(struct_img, window=5)
activity_window_5 = activity.get_activity(active_img, mask)

mask = roi.get_roi(struct_img, window=1)
activity_window_1 = activity.get_activity(active_img, mask)

diff_10 = (activity_window_1 - activity_window_5)/activity_window_1
diff_5 = (activity_window_1 - activity_window_10)/activity_window_1

fig, ax = plt.subplots()

ax.plot(diff_10, 'b.', label='10:1')
ax.plot(diff_5, 'r.', label='5:1')
ax.legend()
fig.suptitle('Percentual error for downsampling')
plt.show()

Activity for both channels:

In [ ]:
i = 2
active_img = imread(os.path.join(img_dir,active[i]))
struct_img = imread(os.path.join(img_dir,struct[i]))

mask = roi.get_roi(struct_img)

activity.plot_activity(active_img, struct_img, mask=mask)

In [ ]:
i = 2
active_img = imread(os.path.join(img_dir,active[i]))
struct_img = imread(os.path.join(img_dir,struct[i]))

filename = active[i][:-8] + '.npy'
full_path = os.path.join(os.path.dirname(os.getcwd()), 'results', 'cache', 'roi', filename)

if not os.path.isfile(full_path):
    roi.cache_rois(struct_img, file_path=full_path)

activity.plot_activity(active_img, struct_img, mask=None, mask_path=full_path, plot_diff=False)